## 1. Background Modeling
These methods estimate the background and detect deviations as foreground.

### 1.1 Gaussian Mixture Models (GMM) – OpenCV Implementation
- Concept: Models each pixel as a mixture of multiple Gaussians.
- Best For: Dynamic backgrounds with periodic changes (e.g., water, trees).

In [1]:
import cv2

# Load video
cap = cv2.VideoCapture("ChangeDetectionDataset\MyDownloads\Road_Video.mp4")

# Initialize Background Subtractor using GMM
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(frame)

    # Display result
    cv2.imshow("Foreground Mask (GMM)", fg_mask)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

### 1.2 Kernel Density Estimation (KDE)
- Concept: Uses non-parametric density estimation to model background.
- Best For: Non-Gaussian backgrounds with complex patterns.

In [3]:
# import numpy as np
# from sklearn.neighbors import KernelDensity
# import cv2

# # Load background image
# background = cv2.imread(r"frames\frame_0004.jpg", 0)  # Grayscale

# # Convert to 1D array for KDE
# background_data = background.flatten().reshape(-1, 1)

# # Fit KDE model
# kde = KernelDensity(kernel="gaussian", bandwidth=5).fit(background_data)

# # Load a new frame
# frame = cv2.imread(r"frames\frame_0187.jpg", 0)

# # Compute likelihood
# frame_data = frame.flatten().reshape(-1, 1)
# likelihood = kde.score_samples(frame_data)

# # Reshape and threshold
# likelihood = likelihood.reshape(frame.shape)
# fg_mask = (likelihood < np.percentile(likelihood, 5)).astype(np.uint8) * 255

# cv2.imshow("Foreground Mask (KDE)", fg_mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

### 1.3 ViBe (Visual Background Extractor)
- Concept: Maintains a background model using pixel-level samples.
- Best For: Robust real-time background subtraction.

In [4]:
#Separate Implementation later

## 2. Frame Differencing
A simple method that computes the absolute difference between consecutive frames.

In [2]:
cap = cv2.VideoCapture("ChangeDetectionDataset\MyDownloads\Road_Video.mp4")

ret, frame1 = cap.read()
frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

while cap.isOpened():
    ret, frame2 = cap.read()
    if not ret:
        break

    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Compute difference
    diff = cv2.absdiff(frame1_gray, frame2_gray)

    # Apply threshold
    _, fg_mask = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

    cv2.imshow("Foreground Mask (Frame Differencing)", fg_mask)

    frame1_gray = frame2_gray  # Update reference frame

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

## 3. Optical Flow-Based Methods

### 3.1 Dense Optical Flow (Farneback)
- Concept: Computes pixel-wise motion between frames.
- Best For: Detecting continuous motion in videos.

In [6]:
import numpy as np
cap = cv2.VideoCapture("ChangeDetectionDataset\MyDownloads\Road_Video.mp4")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
hsv[..., 1] = 255  # Set saturation to maximum

while cap.isOpened():
    ret, frame2 = cap.read()
    if not ret:
        break

    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Compute optical flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2  # Hue represents motion direction
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)  # Value represents motion intensity

    flow_visual = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    cv2.imshow("Optical Flow (Farneback)", flow_visual)

    prvs = next

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

## 4. Deep Learning-Based Background Subtraction
### FgSegNet (Foreground Segmentation Network)
- Concept: Uses deep learning to segment the moving objects from the background.
- Best For: Complex scenes where traditional methods fail.

## 5. Statistical Change Detection
### 5.1 Chi-Square Test for Pixel Distribution Change
- Concept: Compares pixel distributions in a region to detect significant changes.
- Best For: Statistically robust change detection.

In [11]:
import numpy as np
import cv2
from scipy.stats import chi2_contingency

# Load images (grayscale)
frame1 = cv2.imread(r"frames\frame_0194.jpg", cv2.IMREAD_GRAYSCALE)
frame2 = cv2.imread(r"frames\frame_0196.jpg", cv2.IMREAD_GRAYSCALE)

# Compute histograms
hist1, _ = np.histogram(frame1, bins=256, range=(0, 256))
hist2, _ = np.histogram(frame2, bins=256, range=(0, 256))

# Avoid zero-bin issue by adding a small value (Laplace smoothing)
hist1 += 1
hist2 += 1

# Perform Chi-square test
chi2, p, _, _ = chi2_contingency([hist1, hist2])

print(f"Chi-Square Value: {chi2}, p-value: {p}")

if p < 0.05:
    print("Significant Change Detected!")
else:
    print("No Significant Change.")

Chi-Square Value: 302.5934641898606, p-value: 0.02179025040440417
Significant Change Detected!
